In [1]:
import copy
import glob
import gzip
import json
import os
import shutil
import sys
import time
import warnings
import zipfile
from datetime import date

import cdsapi
import h5py
import numpy
import numpy as np
import pandas as pd
import urllib3
import xarray as xr

sys.path.append(os.getcwd() + "/../cds-backend/code/")
import cds_eua4 as eua

warnings.filterwarnings("ignore")

sys.path.insert(0, os.getcwd() + "/../resort/rasotools-master/")
import matplotlib
import matplotlib.font_manager as font_manager
import matplotlib.pylab as plt
import matplotlib.pyplot as maplt
import rasotools
import seaborn

matplotlib.rcParams.update({"font.size": 20})
matplotlib.rcParams["figure.figsize"] = (20, 10)
font = {
    "size": 20,
}
matplotlib.rc("font", **font)

import trajectory as trj

no config found


In [2]:
import cdsapi

c = cdsapi.Client()

r = c.retrieve(
    'insitu-observations-igra-baseline-network',
    {
        'archive_type': 'harmonized_global_radiosonde_archive',
        'format': 'csv-lev.zip',
        'year': '2000',
        'month': '01',
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],
        'area': [
            60, 10, 40,
            20,
        ],
        'variable': [
            'air_temperature', 'eastward_wind_component', 'northward_wind_component',
            'relative_humidity',
        ],
    },
    'download.csv-lev.zip')

2023-07-20 14:24:30,679 INFO Welcome to the CDS
2023-07-20 14:24:30,679 INFO Sending request to https://cds-test.copernicus-climate.eu/api/v2/resources/insitu-observations-igra-baseline-network
2023-07-20 14:24:30,749 INFO Request is completed
2023-07-20 14:24:30,750 INFO Downloading https://cds-test-download-0000.copernicus-climate.eu/cache-compute-0000/cache/data1/adaptor.insitu_reference.retrieve-1689759763.2069223-31328-1-d402cc8a-aac5-40d6-a0ba-2817bded26f5.zip to download.csv-lev.zip (548.2K)
2023-07-20 14:24:30,956 INFO Download rate 2.6M/s                     


In [3]:
tempdir = "./tmp/"
delete_dl = True
with zipfile.ZipFile("download.csv-lev.zip", "r") as zip_ref:
    try:
        os.mkdir(tempdir)
    except:
        pass

    zip_ref.extractall(tempdir)

for i in glob.glob(tempdir + "*"):
    print(i)
    df = pd.read_csv(i, header=16)

# if delete_dl:
#     shutil.rmtree(tempdir)

./tmp/IGRA_H_20000101_20000131_subset_cdm-lev_v1.csv


In [4]:
df

,station_name,radiosonde_code,sensor_model,report_timestamp,actual_time,report_id,longitude,latitude,height_of_station_above_sea_level,air_pressure,air_temperature,relative_humidity,eastward_wind_component,northward_wind_component
0,AUM00011240,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,2000-01-01 03:44:00+00,82731,15.4392,46.9931,340.0,10000,215.65,NaN,6.06,-3.50
1,AUM00011240,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,2000-01-01 03:44:00+00,82731,15.4392,46.9931,340.0,15000,216.85,NaN,16.74,2.95
2,AUM00011240,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,2000-01-01 03:44:00+00,82731,15.4392,46.9931,340.0,20000,216.05,NaN,28.98,7.76
3,AUM00011240,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,2000-01-01 03:44:00+00,82731,15.4392,46.9931,340.0,22800,216.45,1.93,45.11,16.42
4,AUM00011240,61.0,"Vaisala RS80/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,2000-01-01 03:44:00+00,82731,15.4392,46.9931,340.0,25000,219.05,2.14,51.68,18.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44295,GMM00010184,62.0,Vaisala RS80/PCCora (Finland),2000-01-31 12:00:00+00,2000-01-31 10:52:00+00,11949459,13.4075,54.0978,2.0,92500,275.66,80.40,7.14,-27.09
44296,GMM00010184,62.0,Vaisala RS80/PCCora (Finland),2000-01-31 12:00:00+00,2000-01-31 10:52:00+00,11949459,13.4075,54.0978,2.0,97400,279.56,62.01,NaN,NaN
44297,GMM00010184,62.0,Vaisala RS80/PCCora (Finland),2000-01-31 12:00:00+00,2000-01-31 10:52:00+00,11949459,13.4075,54.0978,2.0,100000,281.50,65.21,3.48,-17.32
44298,GMM00010771,62.0,Vaisala RS80/PCCora (Finland),2000-01-09 00:00:00+00,2000-01-08 22:47:00+00,5848365,11.9022,49.4283,417.0,96600,272.85,95.02,NaN,NaN


In [5]:
lat_disp, lon_disp, sec_disp = np.array([np.nan]*len(df)),np.array([np.nan]*len(df)),np.array([np.nan]*len(df))
for station in df.station_name.drop_duplicates():
    df_i = df[df.station_name == station]
    for rid in df_i.report_id.drop_duplicates():
        df_j = df_i[df_i.report_id == rid]
        df_j_cleanded = df_j.sort_values(by='air_pressure', ascending=False).dropna(subset=['air_temperature', 'eastward_wind_component', 'northward_wind_component'])
        if len(df_j_cleanded) > 3:
            
            idx =  df_j_cleanded.index.values
            lat_i, lon_i, sec_i = trj.trajectory(df_j_cleanded.latitude.iloc[0], 
                                                 df_j_cleanded.longitude.iloc[0], 
                                                 df_j_cleanded.eastward_wind_component.values, 
                                                 df_j_cleanded.northward_wind_component.values, 
                                                 df_j_cleanded.air_pressure.values, 
                                                 df_j_cleanded.air_temperature.values
                                                )
            lat_disp[idx] = lat_i
            lon_disp[idx] = lon_i
            sec_disp[idx] = sec_i
    
df['latitude_displacement'] = lat_disp
df['longitude_displacement'] = lon_disp
df['time_displacement'] = sec_disp

In [6]:
df[df.report_id == df.report_id.drop_duplicates().values[3]]

,station_name,radiosonde_code,sensor_model,report_timestamp,actual_time,report_id,longitude,latitude,height_of_station_above_sea_level,air_pressure,air_temperature,relative_humidity,eastward_wind_component,northward_wind_component,latitude_displacement,longitude_displacement,time_displacement
105,DAM00006181,71.0,"Vaisala RS90/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,1999-12-31 23:05:00+00,6177394,12.533,55.767,40.0,1000,198.45,NaN,9.20,-52.19,-0.862467,1.672128,5866.412361
106,DAM00006181,71.0,"Vaisala RS90/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,1999-12-31 23:05:00+00,6177394,12.533,55.767,40.0,1100,195.85,NaN,NaN,NaN,NaN,NaN,NaN
107,DAM00006181,71.0,"Vaisala RS90/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,1999-12-31 23:05:00+00,6177394,12.533,55.767,40.0,1430,199.25,NaN,NaN,NaN,NaN,NaN,NaN
108,DAM00006181,71.0,"Vaisala RS90/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,1999-12-31 23:05:00+00,6177394,12.533,55.767,40.0,1870,194.85,NaN,NaN,NaN,NaN,NaN,NaN
109,DAM00006181,71.0,"Vaisala RS90/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,1999-12-31 23:05:00+00,6177394,12.533,55.767,40.0,2000,195.85,NaN,13.37,-36.74,-0.542846,1.530259,5066.400079
110,DAM00006181,71.0,"Vaisala RS90/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,1999-12-31 23:05:00+00,6177394,12.533,55.767,40.0,3000,202.05,NaN,9.68,-26.59,-0.408509,1.444451,4594.199199
111,DAM00006181,71.0,"Vaisala RS90/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,1999-12-31 23:05:00+00,6177394,12.533,55.767,40.0,3710,205.05,NaN,NaN,NaN,NaN,NaN,NaN
112,DAM00006181,71.0,"Vaisala RS90/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,1999-12-31 23:05:00+00,6177394,12.533,55.767,40.0,3930,203.05,NaN,NaN,NaN,NaN,NaN,NaN
113,DAM00006181,71.0,"Vaisala RS90/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,1999-12-31 23:05:00+00,6177394,12.533,55.767,40.0,4910,204.45,NaN,NaN,NaN,NaN,NaN,NaN
114,DAM00006181,71.0,"Vaisala RS90/Loran/Digicora I, II or Marwin (F...",2000-01-01 00:00:00+00,1999-12-31 23:05:00+00,6177394,12.533,55.767,40.0,5000,205.25,NaN,14.45,-20.64,-0.279277,1.328222,3985.204325


In [10]:
df_interp = df[df.report_id == df.report_id.drop_duplicates().values[3]]
df_interp['latitude_displacement_interpolated'] = np.interp(df_interp.air_pressure, df_interp[np.isfinite(df_interp.latitude_displacement)].air_pressure, df_interp[np.isfinite(df_interp.latitude_displacement)].latitude_displacement)
df_interp['longitude_displacement_interpolated'] = np.interp(df_interp.air_pressure, df_interp[np.isfinite(df_interp.longitude_displacement)].air_pressure, df_interp[np.isfinite(df_interp.longitude_displacement)].longitude_displacement)
df_interp['time_displacement_interpolated'] = np.interp(df_interp.air_pressure, df_interp[np.isfinite(df_interp.time_displacement)].air_pressure, df_interp[np.isfinite(df_interp.time_displacement)].time_displacement)
df_interp[['air_pressure', 'latitude_displacement', 'latitude_displacement_interpolated', 'longitude_displacement', 'longitude_displacement_interpolated', 'time_displacement', 'time_displacement_interpolated']]

,air_pressure,latitude_displacement,latitude_displacement_interpolated,longitude_displacement,longitude_displacement_interpolated,time_displacement,time_displacement_interpolated
105,1000,-0.862467,-0.862467,1.672128,1.672128,5866.412361,5866.412361
106,1100,NaN,-0.830505,NaN,1.657941,NaN,5786.411133
107,1430,NaN,-0.725030,NaN,1.611124,NaN,5522.407080
108,1870,NaN,-0.584397,NaN,1.548702,NaN,5170.401676
109,2000,-0.542846,-0.542846,1.530259,1.530259,5066.400079,5066.400079
110,3000,-0.408509,-0.408509,1.444451,1.444451,4594.199199,4594.199199
111,3710,NaN,-0.362631,NaN,1.403190,NaN,4378.006018
112,3930,NaN,-0.348416,NaN,1.390404,NaN,4311.016582
113,4910,NaN,-0.285092,NaN,1.333452,NaN,4012.609094
114,5000,-0.279277,-0.279277,1.328222,1.328222,3985.204325,3985.204325


In [11]:
year, mon = 2000, 1
files = glob.glob('/mnt/scratch/scratch/leo/scratch/era5/gridded/era5fct.'+str(year)+str(mon).zfill(2)+'*.132.nc')[0]
ds_fc = xr.load_dataset(files)

look_at = df[df.date_time.dt.month == mon]
look_at = np.unique(look_at.date_time)
for i in look_at[:15]:
    var_list = []
    undis_list = []
    cuon_list = []
    cuon_press = []
    
    out_dict = {}
    ds_fc_time = ds_fc.sel(time=i, method='nearest')
    station_lat = df.latitude.iloc[0]
    station_lon = df.longitude.iloc[0]
    lon = station_lon
    if lon < 0:
        lon = 360.+lon
    ds_now = ds_fc_time.interp(latitude=[station_lat], longitude=[lon], method="linear")
    undis_list = np.array(ds_now.v)
    
    for j in np.array(ds_fc_time.level): #10,20,...,1000
        step = find_nearest(df.z_coordinate, j*100)
        input_data_step = df[df.z_coordinate == step]
        station_lat = df.latitude.iloc[0] + np.array(input_data_step.latitude_displacement)[0]
        station_lon = df.longitude.iloc[0] + np.array(input_data_step.longitude_displacement)[0]
        lon = station_lon
        if lon < 0:
            lon = 360.+lon

        
        ds_now = ds_fc_time.interp(latitude=[station_lat], longitude=[lon], method="linear")
        var = ds_now.v.sel(level = j)
        var_list.append(float(var))
        cv = df[np.logical_and(df.date_time == i,  df.z_coordinate == step)].observation_value.values
        cuon_list.append((np.nan if len(cv) == 0 else float(cv[0])))
        cuon_press.append(step)
        
    out_dict['era5_displaced'] = drop_dims(var_list)
    out_dict['era5'] = drop_dims(undis_list)
    out_dict['cuon'] = drop_dims(cuon_list)
    
    out_dict['pressure'] = drop_dims(ds_fc_time.level) * 100.
    out_dict['cuon_pressure']= drop_dims(cuon_press)
    out_df = pd.DataFrame.from_dict(out_dict)


AttributeError: 'DataFrame' object has no attribute 'date_time'